# Bước 4: Xây dựng mô hình

In [346]:
# Tương tự phầnddaacuar bước 3, ta có:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

# Tương tự như step 1
train_df = pd.read_csv("train.csv", index_col="PassengerId")

features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Survived"]


def convert_cat(df, features):
    for feature in features:
        df[feature] = df[feature].astype("category")


convert_cat(train_df, features)

train_df.info()

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")

X = train_df.iloc[:, :].values

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer.fit(X[:, 3:4])
X[:, 3:4] = imputer.transform(X[:, 3:4])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder="passthrough")
X = ct.fit_transform(X)

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [10])], remainder="passthrough")
X = ct.fit_transform(X)
X

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Pclass    891 non-null    category
 1   Name      891 non-null    object  
 2   Sex       891 non-null    category
 3   Age       714 non-null    float64 
 4   SibSp     891 non-null    category
 5   Parch     891 non-null    category
 6   Ticket    891 non-null    object  
 7   Fare      891 non-null    float64 
 8   Cabin     204 non-null    object  
 9   Embarked  889 non-null    category
 10  Survived  891 non-null    category
dtypes: category(6), float64(2), object(3)
memory usage: 48.2+ KB


array([[0.0, 0.0, 1.0, ..., 7.25, nan, 0],
       [1.0, 0.0, 0.0, ..., 71.2833, 'C85', 1],
       [0.0, 0.0, 1.0, ..., 7.925, nan, 1],
       ...,
       [0.0, 0.0, 1.0, ..., 23.45, nan, 0],
       [1.0, 0.0, 0.0, ..., 30.0, 'C148', 1],
       [0.0, 1.0, 0.0, ..., 7.75, nan, 0]], dtype=object)

In [347]:
# Convert X from data to data frame
data_df = pd.DataFrame(X)

In [348]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,0.0,1.0,0.0,0.0,1.0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.25,NaN,0
1,1.0,0.0,0.0,0.0,1.0,0.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1
2,0.0,0.0,1.0,0.0,1.0,0.0,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.925,NaN,1
3,0.0,0.0,1.0,0.0,1.0,0.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1,C123,1
4,0.0,0.0,1.0,0.0,0.0,1.0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.05,NaN,0


- Vì không phải tất cả các features đều có giá trị (như tên, địa chỉ nhà, ...) , nên ta cần cho các feature cần thiết

In [349]:
# Select features
features = [0, 1, 2, 3, 4, 5, 6, 8, 9, 10,12]

X = data_df[features]
X


,0,1,2,3,4,5,6,8,9,10,12
0,0.0,0.0,1.0,0.0,0.0,1.0,3,22.0,1,0,7.25
1,1.0,0.0,0.0,0.0,1.0,0.0,1,38.0,1,0,71.2833
2,0.0,0.0,1.0,0.0,1.0,0.0,3,26.0,0,0,7.925
3,0.0,0.0,1.0,0.0,1.0,0.0,1,35.0,1,0,53.1
4,0.0,0.0,1.0,0.0,0.0,1.0,3,35.0,0,0,8.05
...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,0.0,1.0,0.0,0.0,1.0,2,27.0,0,0,13.0
887,0.0,0.0,1.0,0.0,1.0,0.0,1,19.0,0,0,30.0
888,0.0,0.0,1.0,0.0,1.0,0.0,3,29.699118,1,2,23.45
889,1.0,0.0,0.0,0.0,0.0,1.0,1,26.0,0,0,30.0


In [350]:
features = [14]
y = data_df[features]

y[14] = y[14].astype("category")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14500\3815402436.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[14] = y[14].astype("category")


### Phân chia dữ liệu thành training set và test set

In [351]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

## Training Machine Learning Model

### Ở đây chúng ta sử dụng DecisionTreeClassifier (phân cụm, vì chỉ cần biết người đó có sống hay không)

In [352]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=1)

In [353]:
# cài đặt model gồm X_train và y_train
dt_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=1)

In [354]:
# đưa X test vào model đã dựng
y_preds = dt_model.predict(X_valid)

In [355]:
# chuyển về cùng dạng array
y_preds = pd.array(y_preds)

y_valid = pd.array(y_valid[14])

In [356]:
pd.DataFrame({'y': y_valid, 'y_preds': y_preds})

,y,y_preds
0,0,0
1,0,0
2,0,0
3,1,1
4,1,0
...,...,...
174,1,1
175,0,0
176,1,1
177,0,0


### Tỉ lệ chính xác ~78%